<a href="https://colab.research.google.com/github/pratikshaa12/NLP/blob/main/Solution%2BLexicon%2BRule_based%2BPOS%2BTagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reading and understanding the tagged dataset

In [8]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import pprint,time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import math

In [9]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [10]:
# Reading the treebank tagged sentences
corpus = list(nltk.corpus.treebank.tagged_sents())

In [11]:
# Samples: each sentence is a list of (word, pos) tuples
corpus[:3]

[[('Pierre', 'NNP'),
  ('Vinken', 'NNP'),
  (',', ','),
  ('61', 'CD'),
  ('years', 'NNS'),
  ('old', 'JJ'),
  (',', ','),
  ('will', 'MD'),
  ('join', 'VB'),
  ('the', 'DT'),
  ('board', 'NN'),
  ('as', 'IN'),
  ('a', 'DT'),
  ('nonexecutive', 'JJ'),
  ('director', 'NN'),
  ('Nov.', 'NNP'),
  ('29', 'CD'),
  ('.', '.')],
 [('Mr.', 'NNP'),
  ('Vinken', 'NNP'),
  ('is', 'VBZ'),
  ('chairman', 'NN'),
  ('of', 'IN'),
  ('Elsevier', 'NNP'),
  ('N.V.', 'NNP'),
  (',', ','),
  ('the', 'DT'),
  ('Dutch', 'NNP'),
  ('publishing', 'VBG'),
  ('group', 'NN'),
  ('.', '.')],
 [('Rudolph', 'NNP'),
  ('Agnew', 'NNP'),
  (',', ','),
  ('55', 'CD'),
  ('years', 'NNS'),
  ('old', 'JJ'),
  ('and', 'CC'),
  ('former', 'JJ'),
  ('chairman', 'NN'),
  ('of', 'IN'),
  ('Consolidated', 'NNP'),
  ('Gold', 'NNP'),
  ('Fields', 'NNP'),
  ('PLC', 'NNP'),
  (',', ','),
  ('was', 'VBD'),
  ('named', 'VBN'),
  ('*-1', '-NONE-'),
  ('a', 'DT'),
  ('nonexecutive', 'JJ'),
  ('director', 'NN'),
  ('of', 'IN'),
  ('this'

In [12]:
# Convertting the list of sents to a list of(word, pos tag) tuples
tagged_words = [tup for sent in corpus for tup in sent]
print(len(tagged_words))
tagged_words[:50]

100676


[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.'),
 ('Mr.', 'NNP'),
 ('Vinken', 'NNP'),
 ('is', 'VBZ'),
 ('chairman', 'NN'),
 ('of', 'IN'),
 ('Elsevier', 'NNP'),
 ('N.V.', 'NNP'),
 (',', ','),
 ('the', 'DT'),
 ('Dutch', 'NNP'),
 ('publishing', 'VBG'),
 ('group', 'NN'),
 ('.', '.'),
 ('Rudolph', 'NNP'),
 ('Agnew', 'NNP'),
 (',', ','),
 ('55', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 ('and', 'CC'),
 ('former', 'JJ'),
 ('chairman', 'NN'),
 ('of', 'IN'),
 ('Consolidated', 'NNP'),
 ('Gold', 'NNP'),
 ('Fields', 'NNP'),
 ('PLC', 'NNP'),
 (',', ','),
 ('was', 'VBD'),
 ('named', 'VBN'),
 ('*-1', '-NONE-'),
 ('a', 'DT')]

### Exploratory Analysis

In [13]:
# Find the number of unique pos tags in the corpus
# u can use the set() function on the list of tags to get a unique set of tags,
# compute its length

tags = [pair[1] for pair in tagged_words]
unique_tags = set(tags)
len(unique_tags)

46

In [14]:
# which is the most frequent tag in the corpus
# to count the frequency of elements in a list , the counter() class from collections
# module is very useful, as shown below

from collections import Counter
tag_counts = Counter(tags)
tag_counts

Counter({'NNP': 9410,
         ',': 4886,
         'CD': 3546,
         'NNS': 6047,
         'JJ': 5834,
         'MD': 927,
         'VB': 2554,
         'DT': 8165,
         'NN': 13166,
         'IN': 9857,
         '.': 3874,
         'VBZ': 2125,
         'VBG': 1460,
         'CC': 2265,
         'VBD': 3043,
         'VBN': 2134,
         '-NONE-': 6592,
         'RB': 2822,
         'TO': 2179,
         'PRP': 1716,
         'RBR': 136,
         'WDT': 445,
         'VBP': 1321,
         'RP': 216,
         'PRP$': 766,
         'JJS': 182,
         'POS': 824,
         '``': 712,
         'EX': 88,
         "''": 694,
         'WP': 241,
         ':': 563,
         'JJR': 381,
         'WRB': 178,
         '$': 724,
         'NNPS': 244,
         'WP$': 14,
         '-LRB-': 120,
         '-RRB-': 126,
         'PDT': 27,
         'RBS': 35,
         'FW': 4,
         'UH': 3,
         'SYM': 1,
         'LS': 13,
         '#': 16})

In [15]:
# the most common tags can be seen using the most_common() method of counter
tag_counts.most_common(5)

[('NN', 13166), ('IN', 9857), ('NNP', 9410), ('DT', 8165), ('-NONE-', 6592)]

In [16]:
import nltk
nltk.download('tagsets_json')

[nltk_data] Downloading package tagsets_json to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets_json.zip.


True

In [17]:
# list of pos tags in NLTK
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [18]:
# which tag is most commonly assigned to the word w.
bank = [pair for pair in tagged_words if pair[0].lower() == 'bank']
bank

[('bank', 'NN'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('Bank', 'NNP'),
 ('bank', 'NN'),
 ('bank', 'NN'),
 ('Bank', 'NNP'),
 

In [19]:
# which tag is most commonly assigned to the word w.
executive = [pair for pair in tagged_words if pair[0].lower() == 'executive']
executive

[('executive', 'NN'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'NN'),
 ('executive', 'JJ'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'JJ'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'NN'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executive', 'JJ'),
 ('executi

In [20]:
# how many words with the tag 'VBG' end with 'ing'
participle_verbs = [pair for pair in tagged_words if pair[1]=='VBG']
ing_verbs =[pair for pair in participle_verbs if pair[0].endswith('ing')]
print(len(ing_verbs) / len(participle_verbs))
ing_verbs[:20]

0.9972602739726028


[('publishing', 'VBG'),
 ('causing', 'VBG'),
 ('using', 'VBG'),
 ('talking', 'VBG'),
 ('having', 'VBG'),
 ('making', 'VBG'),
 ('surviving', 'VBG'),
 ('including', 'VBG'),
 ('including', 'VBG'),
 ('according', 'VBG'),
 ('remaining', 'VBG'),
 ('according', 'VBG'),
 ('declining', 'VBG'),
 ('rising', 'VBG'),
 ('yielding', 'VBG'),
 ('waiving', 'VBG'),
 ('holding', 'VBG'),
 ('holding', 'VBG'),
 ('cutting', 'VBG'),
 ('manufacturing', 'VBG')]

In [21]:
# how many words with the tag 'VBD' (verb, past tense) end with 'ed'
past_tense_verbs = [pair for pair in tagged_words if pair[1]=='VBD']
ed_verbs =[pair for pair in past_tense_verbs if pair[0].endswith('ed')]
print(len(ed_verbs) / len(past_tense_verbs))
ed_verbs[:20]

0.3881038448899113


[('reported', 'VBD'),
 ('stopped', 'VBD'),
 ('studied', 'VBD'),
 ('led', 'VBD'),
 ('worked', 'VBD'),
 ('explained', 'VBD'),
 ('imposed', 'VBD'),
 ('dumped', 'VBD'),
 ('poured', 'VBD'),
 ('mixed', 'VBD'),
 ('described', 'VBD'),
 ('ventilated', 'VBD'),
 ('contracted', 'VBD'),
 ('continued', 'VBD'),
 ('eased', 'VBD'),
 ('ended', 'VBD'),
 ('lengthened', 'VBD'),
 ('reached', 'VBD'),
 ('resigned', 'VBD'),
 ('approved', 'VBD')]

In [22]:
# what fraction of adjectives JJ are followed by a noun NN
# create a list of all tags (without the words )
tags = [pair[1] for pair in tagged_words]

# create list of JJ tags
jj_tags = [ t for t in tags if t =='JJ']

# create  a list of (JJ, NN) tags
jj_nn_tags = [(t, tags[index+1]) for index, t in enumerate(tags)
               if t=='JJ' and tags[index+1]=='NN']
print(len(jj_tags))
print(len(jj_nn_tags))
print(len(jj_nn_tags) / len(jj_tags))


5834
2611
0.4475488515598217


In [23]:
 #what fraction of adjectives JJ are followed by a noun NN
# create a list of all tags (without the words )
tags = [pair[1] for pair in tagged_words]

# create list of DT tags
dt_tags = [ t for t in tags if t =='DT']

# create  a list of (DT, NN) tags
dt_nn_tags = [(t, tags[index+1]) for index, t in enumerate(tags)
               if t=='DT' and tags[index+1]=='NN']
print(len(dt_tags))
print(len(dt_nn_tags))
print(len(dt_nn_tags) / len(dt_tags))


8165
3844
0.470789957134109


In [24]:
 #what fraction of models MD   are followed by a verb VB


# create list of MD tags
md_tags = [ t for t in tags if t =='MD']

# create  a list of (MD, VB) tags
md_vb_tags = [(t, tags[index+1]) for index, t in enumerate(tags)
               if t=='MD' and tags[index+1]=='VB']
print(len(md_tags))
print(len(md_vb_tags))
print(len(md_vb_tags) / len(md_tags))


927
756
0.8155339805825242


## Lexicon and Rule-Based Models for POS Tagging

##### We'll first split the corpus into training and test sets and then use built-in NLTK taggers

#### Splitting into Train and Test Sets

In [25]:
# splitting into train and test
random.seed(1234)
train_set, test_set =train_test_split(corpus, test_size=0.3)
print(len(train_set))
print(len(test_set))
print(train_set[:2])

2739
1175
[[('But', 'CC'), ('its', 'PRP$'), ('17', 'CD'), ('big', 'JJ'), ('junk', 'NN'), ('holdings', 'NNS'), ('at', 'IN'), ('year', 'NN'), ('end', 'NN'), ('showed', 'VBD'), ('only', 'RB'), ('a', 'DT'), ('few', 'JJ'), ('bonds', 'NNS'), ('that', 'WDT'), ('*T*-1', '-NONE-'), ('have', 'VBP'), ('been', 'VBN'), ('really', 'RB'), ('battered', 'VBN'), ('*-2', '-NONE-'), ('.', '.')], [('But', 'CC'), ('GMAC', 'NNP'), ('approved', 'VBD'), ('the', 'DT'), ('Buick', 'NNP'), ('program', 'NN'), (',', ','), ('he', 'PRP'), ('says', 'VBZ'), ('0', '-NONE-'), ('*T*-1', '-NONE-'), (',', ','), ('because', 'IN'), ('the', 'DT'), ('American', 'NNP'), ('Express', 'NNP'), ('green', 'JJ'), ('card', 'NN'), ('requires', 'VBZ'), ('payment', 'NN'), ('in', 'IN'), ('full', 'JJ'), ('upon', 'IN'), ('billing', 'NN'), (',', ','), ('and', 'CC'), ('so', 'RB'), ('does', 'VBZ'), ("n't", 'RB'), ('carry', 'VB'), ('any', 'DT'), ('finance', 'NN'), ('rates', 'NNS'), ('.', '.')]]


### Lexicon(Unigram) Tagger

##### let's  now try training a lexicon tagger which assigns the most commonly assigned tag to a word. In nltk UnigramTagger() can be used to train such a model

In [26]:
# Lexicon or unigram tagger
unigram_tagger = nltk.UnigramTagger(train_set)
unigram_tagger.evaluate(test_set)

<ipython-input-26-d71c97a7a54c>:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  unigram_tagger.evaluate(test_set)


0.8744598739980869

### Rule-Based (Regular Expression) Tagger

##### In nltk, the RegexpTagger() can be provided with handwritten regular expresion patterns.

In [27]:
# specify patterns for tagging
# example from the nltk book
patterns = [
    (r'.*ing$', 'VBG'),         #gerund
    (r'. *ed$', 'VBD'),         # past tense
    (r'. *es$', 'VBZ'),         # 3rd singular present
    (r'.*ould$', 'MD'),         # modals
    (r'.*\'s$', 'NN$'),         # possessive niuns
    (r'.*s$', 'NNS'),           # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'), # cardinal numbers
    (r'.*', 'NN')                     # nouns
]

In [28]:
regexp_tagger = nltk.RegexpTagger(patterns)
# help(regexp_Tagger)

In [29]:
regexp_tagger.evaluate(test_set)

<ipython-input-29-84af8b614d0b>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  regexp_tagger.evaluate(test_set)


0.2157205528251476

### Combining Taggers

In [30]:
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

# lexicon backrd up bu the rule based tagger
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)
lexicon_tagger.evaluate(test_set)

<ipython-input-30-8d3dd2ab4abc>:6: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  lexicon_tagger.evaluate(test_set)


0.9063561698057195